- 회귀
- 손실함수 : mean_squared_error
- 출력층의 노드의 개수 : 예측할 값의 종류 수
- 회귀는 손실율이 초반에 굉장히 크게 나타날 수 있다.
- 그렇다고 해서 데이터 학습을 잘 못하는 것이 아니고 초기에 패턴을 맞출때는 패턴에 대한 학습은 잘 이루어지지만 실제값과 차이가 발생하는 경우에 해당한다.
- 이럴 때는 학습 횟수를 크게 증가시켜서 학습을 수행해야 한다.
- 만약 학습 횟수를 줄이고자 한다면 결과 데이터를 MinMaxScaler로 스케일링 하고 출력층의 활성화 함수를 sigmoid로 설정해 준다.

In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False
# 매직명령어 => 쥬피터노트북에서 그래프 삽입 기능 
%matplotlib inline
# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
# 신경망 모델을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형 회귀 레이어
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 객체
from tensorflow.keras.layers import Activation
# 원핫 인코딩을 수행하는 함수
from tensorflow.keras.utils import to_categorical

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 문자열 => 숫자
from sklearn.preprocessing import LabelEncoder

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [2]:
# 데이터를 읽어온다.
# 이 파일은 띄어쓰기로 구분되어 있다.
df1 = pd.read_csv('./data/housing.csv', header=None, delim_whitespace=True)
df1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
# 전반적인 정보를 확인한다.
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       506 non-null    float64
 1   1       506 non-null    float64
 2   2       506 non-null    float64
 3   3       506 non-null    int64  
 4   4       506 non-null    float64
 5   5       506 non-null    float64
 6   6       506 non-null    float64
 7   7       506 non-null    float64
 8   8       506 non-null    int64  
 9   9       506 non-null    float64
 10  10      506 non-null    float64
 11  11      506 non-null    float64
 12  12      506 non-null    float64
 13  13      506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [4]:
# 결측치 확인
df1.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [5]:
# 입력과 결과로 나눈다.
X = df1.drop(13, axis=1)
y = df1[13]

In [6]:
# 2진 분류 옵션들
# out_nodes = 1
# loss_function = 'binary_crossentropy'
# activation_function = 'sigmoid'

# 다중 분류 옵션들
# out_nodes = len(y.value_counts())
# loss_function = 'categorical_crossentropy'
# activation_function = 'softmax'

# 회귀
# 출력 결과를 하나만 뽑아서 Series로 나왔다면...
# 1로 설정한다.
if type(y) is pd.core.series.Series:
    out_nodes = 1
else : 
    out_nodes = y.shape[1]
    
loss_function = 'mean_squared_error'

In [7]:
# 표준화
scaler1 = StandardScaler()
X = scaler1.fit_transform(X)
display(X)

y = y.values.reshape(-1, 1)
scaler2 = MinMaxScaler()
y = scaler2.fit_transform(y)
y = y.reshape(-1)
y

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

array([0.42222222, 0.36888889, 0.66      , 0.63111111, 0.69333333,
       0.52666667, 0.39777778, 0.49111111, 0.25555556, 0.30888889,
       0.22222222, 0.30888889, 0.37111111, 0.34222222, 0.29333333,
       0.33111111, 0.40222222, 0.27777778, 0.33777778, 0.29333333,
       0.19111111, 0.32444444, 0.22666667, 0.21111111, 0.23555556,
       0.19777778, 0.25777778, 0.21777778, 0.29777778, 0.35555556,
       0.17111111, 0.21111111, 0.18222222, 0.18      , 0.18888889,
       0.30888889, 0.33333333, 0.35555556, 0.43777778, 0.57333333,
       0.66444444, 0.48      , 0.45111111, 0.43777778, 0.36      ,
       0.31777778, 0.33333333, 0.25777778, 0.20888889, 0.32      ,
       0.32666667, 0.34444444, 0.44444444, 0.40888889, 0.30888889,
       0.67555556, 0.43777778, 0.59111111, 0.40666667, 0.32444444,
       0.30444444, 0.24444444, 0.38222222, 0.44444444, 0.62222222,
       0.41111111, 0.32      , 0.37777778, 0.27555556, 0.35333333,
       0.42666667, 0.37111111, 0.39555556, 0.40888889, 0.42444

In [8]:
# 입력노드의 개수
# 입력 데이터 행 하나의 컬럼의 개수
input_size = X.shape[1]
input_size

13

In [9]:
# 신경망 설계
model = Sequential()

model.add(Dense(12, input_dim=input_size))
model.add(Activation('relu'))

model.add(Dense(out_nodes))
model.add(Activation('sigmoid'))

In [10]:
# 모델 컴파일
model.compile(loss=loss_function, optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                168       
                                                                 
 activation (Activation)     (None, 12)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 13        
                                                                 
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [11]:
# 학습
model.fit(X, y, epochs=200, batch_size=10)

Epoch 1/200
51/51 [==============================] - 1s 2ms/step - loss: 0.2354
Epoch 2/200
51/51 [==============================] - 0s 2ms/step - loss: 0.1283
Epoch 3/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0819
Epoch 4/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0564
Epoch 5/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0411
Epoch 6/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0323
Epoch 7/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0270
Epoch 8/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0231
Epoch 9/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0203
Epoch 10/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 11/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0166
Epoch 12/200
51/51 [==============================] - 0s 2ms/step - loss: 0.0152
Epoch 13/200
51/51 [=================

In [12]:
# 정확도 확인
a1 = model.evaluate(X, y)
print(f'손실률 : {a1}')

16/16 [==============================] - 0s 2ms/step - loss: 0.0031
손실률 : 0.0031234470661729574
